<a href="https://colab.research.google.com/github/Viny2030/UNED/blob/main/TP1_Analisis_de_sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
smid80_coronavirus_covid19_tweets_late_april_path = kagglehub.dataset_download('smid80/coronavirus-covid19-tweets-late-april')

print('Data source import complete.')


100%|██████████| 946M/946M [00:20<00:00, 48.1MB/s]

Extracting files...


Data source import complete.


# TP1 - Analisis de sentimientos
Vamos a realizar un análisis de sentimiento en base a los tweets obtenidos de la segunda quincena de abril de 2020 sobre el covid19.

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import re
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Unión y selección de datos
Lo primero que hacemos es unir todos los csv para poder procesarlos en un único archivo. Vamos a usar DataFrame de Pandas.

In [ ]:
import glob
path = '/kaggle/input/coronavirus-covid19-tweets-late-april/'
all_files = glob.glob(os.path.join(path, "*.CSV"))

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

* Nos quedamos únicamente con la columna del texto correspondiente al tweet, y del lenguaje:

In [ ]:
df = concatenated_df[['text', 'lang']]

* Luego nos quedamos solo con los tweets que están en ingles ("en"):

In [ ]:
filter = df.mask(lambda x: x['lang'] != 'en').dropna()
filter.head()

* Eliminamos los tweet duplicados:

In [ ]:
texts = filter['text'].drop_duplicates()

## Clasificación de los datos
En este punto ya tenemos en nuestro dataset 3.260.790 tweets.

A continuación vamos a extraer los que en sus textos contienen los iconos que representan los sentimientos para clasificar.

* Comenzamos con los de "Alegría":

In [ ]:
joy_icons = '😁|😂|😃|😄|😅|😆|😉|😊|😍'

joy = pd.Series(texts, dtype="string", name="joy").str.contains(joy_icons)
joy

* Continuamos con los de "tristeza - miedo":

In [ ]:
sad_icons = '😓|😖|😢|😭|😰|😱|🙍|🙎'

sad = pd.Series(texts, dtype="string", name="sad").str.contains(sad_icons)
sad

* Continuamos con los de "enojo":

In [ ]:
angry_icons = '😠|😡|😤|🤬|👿|💀|☠'

angry = pd.Series(texts, dtype="string", name="angry").str.contains(angry_icons)
angry

* Y terminamos con los de "apoyo - esperanza":

In [ ]:
hope_icons = '😷|🙋|🙌|🙏'

hope = pd.Series(texts, dtype="string", name="hope").str.contains(hope_icons)
hope

* Unimos los resultados en un nuevo data frame:

In [ ]:
df = pd.concat([texts, joy, sad, angry, hope], axis=1)
df

* Ahora dividimos el dataset en 2, por un lado los que tienen alguna categoría, y por otro los que no tienen ninguna:

In [ ]:
texts_classified = df[(df['joy'] | df['sad'] | df['angry'] | df['hope'])]
texts_not_classified = df[~(df['joy'] | df['sad'] | df['angry'] | df['hope'])]['text']

* Con los que estan clasificados, tenemos que limpiar los que tienen más de 1 categoría. Para eso vamos a sumar la cantidad de categorias en una nueva columna y quedarnos unicamente con los que tienen 1:

In [ ]:
count = texts_classified.apply(lambda row: row['joy'] + row['sad'] + row['angry'] + row['hope'], axis=1)
count = count.rename("count")

In [ ]:
texts_classified = pd.concat([texts_classified, count], axis=1)

In [ ]:
texts_classified = texts_classified[texts_classified['count'] == 1]
texts_classified

* Ahora en base a lo obtenido, vamos a etiquetar según qué categoría sean, los resultados irán en la columna 'target':

In [ ]:
def label_feeling (row):
   if row['joy'] :
      return 'JOY'
   if row['sad'] :
      return 'SAD'
   if row['angry'] :
      return 'ANGRY'
   if row['hope'] :
      return 'HOPE'
   return 'NONE'

texts_classified['target'] = texts_classified.apply(lambda row: label_feeling(row), axis=1)
texts_classified

* Y finalmente nos quedamos solo con las columnas 'text' y 'target', así nos queda nuestro set de datos clasificado:

In [ ]:
texts_classified = texts_classified[['text', 'target']]
texts_classified.head()

## Limpieza de datos
Antes de entrenar nuestro algoritmo, vamos a hacer una limpieza de datos a los textos de los tweets

* Primero vamos a aplicar una limpieza con regex para dejar solo las palabras sin puntuaciones ni números:

In [ ]:
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))

    # remove numbers
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"\d+", "", elem))

    return df

In [ ]:
clean_text(texts_classified, 'text')
texts_classified.head()

* Luego vamos a eliminar todas las "stop words" para que queden las palabras más importantes:

In [ ]:
stop = stopwords.words('english')

texts_classified['text'] = texts_classified['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
texts_classified

## Entrenamiento
Vamos ahora a pasar a la etapa de entrenamiento, a continuación detallamos los valores actuales

In [ ]:
texts_classified['target'].value_counts()

El dataset clasificado cuenta con 104.232 tweets. De los cuales tenemos:

* HOPE con 48.484
* JOY con 36.174
* SAD con 10.660
* ANGRY con 8.914

Como vemos, los datos están desbalanceados, entonces procedemos a acortar el dataset para poder hacer un entrenamiento balanceado.

Tomamos como número màximo el de **ANGRY** ya que es el más chico:

In [ ]:
hope_target = texts_classified[texts_classified['target'] == 'HOPE'].sample(8914)
joy_target = texts_classified[texts_classified['target'] == 'JOY'].sample(8914)
sad_target = texts_classified[texts_classified['target'] == 'SAD'].sample(8914)
angry_target = texts_classified[texts_classified['target'] == 'ANGRY'].sample(8914)

In [ ]:
balanced_data = pd.concat([hope_target, joy_target, sad_target, angry_target], ignore_index=True)
balanced_data

* Con este dataset reducido, dividimos en datos de entrenamiento (66%) y datos de prueba (33%), sin perder el balance (para eso se usa stratify):

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(balanced_data['text'], balanced_data['target'], test_size=0.33, random_state=0, stratify=balanced_data['target'])

* Preparamos el pipeline con el algoritmo de Naive Bayes:

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

* Y lo entrenamos con el dataset de entrenamiento:

In [ ]:
text_clf = text_clf.fit(X_train, y_train)

* Finalmente, predecimos en base a los datos de prueba y evaluamos los resultados:

In [ ]:
predicted = text_clf.predict(X_test)
print(classification_report(y_test, predicted))

* Y así obtenemos los siguientes resultados:

                precision    recall  f1-score   support

       ANGRY       0.33      0.82      0.47      2942
        HOPE       0.81      0.61      0.70     16000
         JOY       0.71      0.50      0.58     11937
         SAD       0.35      0.66      0.45      3518

    accuracy                           0.59     34397
    macro avg      0.55      0.65      0.55     34397
    weighted avg   0.69      0.59      0.61     34397

Como conclusión de este entrenamiento, podemos decir que no son los resultados que esperabamos, en el medio se intentaron diversas tecnicas de limpieza de datos que no aportaron mucha mejora. El problema base recae en la cantidad de datos usados para el entrenamiento, ya que el tener que rebalancear, me vi obligado a desprenderme de ciertos datos ya clasificados. La tendencia de clases al rebalancear termina siendo distinta a la obtenida a partir de los emojis, esto también puede influir cuando pasemos a la predicción de los tweets sin clasificar.

La ditribución nos quedo:
* HOPE con 11996
* JOY con 8334
* ANGRY con 7335
* SAD con 6732

In [ ]:
pd.Series(predicted).value_counts()

## Predicción
Pasamos a predecir los tweets que nos quedaron sin clasificar. En total son 3.150.070.

* Primero que nada, limpiamos los datos de igual manera que los de entrenamiento:

In [ ]:
clean_data = texts_not_classified.str.lower()
clean_data = clean_data.apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
clean_data = clean_data.apply(lambda elem: re.sub(r"\d+", "", elem))
clean_data

In [ ]:
clean_data = clean_data.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
clean_data

* Una vez preparados, pasamos a la predicción, utilizando el algoritmo ya entrena previamente:

In [ ]:
results = text_clf.predict(clean_data)

* Y vemos los resultados;

In [ ]:
pd.Series(results).value_counts()

Los resultados finales son:

* ANGRY con 1.169.389 (37,12%)
* HOPE con 1.042.784 (33,1%)
* SAD con 528.640 (16,78%)
* JOY con 409.257 (12,99%)

Los tweets positivos representan el 46.1%, mientras que los negativos el 53,9%

A primera vista, si comparamos con el balance de los tweets clasificados, no hay correlación. En estos, hay mayor tendencia hacia a los negativos, en cambio en los de entrenamiento era hacia los positivos. Ahora vamos a comparar contra otro algoritmo.

# Parte 2
Para esta parte utilizamos el algoritmo sacado de la página https://nlpforhackers.io/sentiment-analysis-intro/ que utiliza la libreria de SentiWordNet de Natural Language ToolKits.

Previo a eso hace un preprocesamiento de los datos, con lematización de por medio.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag


lemmatizer = WordNetLemmatizer()


def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


def clean_text(text):
    text = text.replace("<br />", " ")

    return text


def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """

    sentiment = 0.0
    tokens_count = 0

    text = clean_text(text)


    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())

            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1

    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0

    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1

    # negative sentiment
    return 0

* Corremos el algoritmo (no recomiendo correr de nuevo, ya que no está optimizado y tomó mucho tiempo llegar a resultados):

In [ ]:
pred_y = [swn_polarity(text) for text in clean_data]

In [ ]:
pd.Series(pred_y).value_counts()

Los resultados obtenidos fueron:

* Positivos: 2.205.853 (70,03%)
* Negativos: 944.217 (29.97%)

Como podemos ver, los datos obtenidos con el SentiWordNet, tienen mucha mas correlación con los obtenidos y clasificados a partir de los emojis. Esto quiere decir, que estabamos en lo correcto cuando deciamos que nuestro algoritmo de naive bayes no nos había dado muy buenos resultados, a pesar de tener una precisión cercana al 70% en la prueba.

## Conclusión
A pesar de no poder lograr mejorar el algoritmo de naive bayes, siento que se hizo un buen trabajo con el manejo y limpieza de datos. Quizas se tendría que haber profundizado más en la identificación de elementos dentro de los tweets. Más allá de eso, los resultados a primera vista no parecían ser tan malos, las comparaciones finales demostraron lo contrario.

Podemos concluir que una de las causas raices de la mala performance fue la baja cantidad de datos negativos más que nada. Eso causo un desbalanceo en los datos de entrenamiento. A pesa de que agregamos emojis, no pudimos subir los porcentajes de clases dentro de los clasificados. Esto nos llevo a reducir datos de entrenamiento para poder entrenar el algoritmo de manera balanceada.

Como mejora a futuro, probaría correr de vuelta el algoritmo pero sin rebalancear las clases, teniendo en cuenta que es muy probable que la tendencia que sigan los tweets sin clasificar se correlacione con los clasificados. Esto quedó demostrado en parte al correr el algoritmo de SentiWordNet.